<a href="https://colab.research.google.com/github/danielbehargithub/MuskTweets-Impact-on-TeslaStock/blob/main/Cleaning_tweets_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
import pandas as pd
from textblob import TextBlob
import numpy as np
import time
import re
from sklearn.preprocessing import MinMaxScaler

In [ ]:
tweets_df = pd.read_csv('/content/elon_musk_tweets.csv')
for i in range(len(tweets_df['text'])):
  tweets_df.text[i] = re.sub(r'\bhttp\S*', '', tweets_df.text[i])
tweets_df

In [ ]:
# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define keyword categories
primary_keywords = ['Tesla', 'TSLA', 'Elon Musk', 'Model S', 'Model 3', 'Model X',
                    'Model Y', 'Cybertruck', 'Roadster', 'Semi', 'Autopilot',
                    'Full Self-Driving', 'Gigafactory', 'Solar Roof', 'Powerwall', 'Starlink']

secondary_keywords = ['@Tesla', 'Electric Vehicle', 'Battery', 'Lithium-ion', 'Renewable Energy',
                      'Autonomous Driving', 'Clean Energy', 'Sustainable Energy',
                      'Energy Storage', 'Solar Energy', 'Charging Infrastructure',
                      'Supercharger', 'AI', 'Machine Learning', 'Autonomous Vehicles',
                      'Green Technology', 'Smart Grid']

financial_keywords = ['Stock', 'NASDAQ', 'Earnings', 'Share', 'Dividend', 'Market Cap',
                      'Valuation', 'Quarterly Report', 'Profit', 'Revenue', 'EPS',
                      'Buyback', 'Stock Split', 'Shareholders', 'Financial Results',
                      'Forecast', 'Guidance', 'Analyst Rating', 'Price Target',
                      'Upgrade', 'Downgrade', 'SEC Filing', 'IPO', 'Mergers and Acquisitions',
                      'Investment', 'Contract']

# Keyword matching scores
tweets_df['primary_keyword_score'] = tweets_df['text'].apply(lambda x: sum(1 for kw in primary_keywords if kw.lower() in x.lower()))
tweets_df['secondary_keyword_score'] = tweets_df['text'].apply(lambda x: sum(1 for kw in secondary_keywords if kw.lower() in x.lower()))

# Update financial_keyword_score to 0 if both primary and secondary scores are 0
tweets_df['financial_keyword_score'] = tweets_df.apply(
    lambda row: sum(1 for kw in financial_keywords if kw.lower() in row['text'].lower()) if row['primary_keyword_score'] > 0 or row['secondary_keyword_score'] > 0 else 0,
    axis=1
)

# Total keyword score with different weights for each category
tweets_df['relevance_to_tesla'] = (tweets_df['primary_keyword_score'] * 5) + (tweets_df['secondary_keyword_score'] * 2)
tweets_df['relevance_to_finance'] = tweets_df['financial_keyword_score']

In [ ]:
tweets_df[['text', 'relevance_to_tesla', 'relevance_to_finance']]

,text,relevance_to_tesla,relevance_to_finance
0,@BillyM2k I find the gold toe sock – inevitabl...,0,0
1,"Sock Con, the conference for socks",0,0
2,Always something new for the magazine cover an...,0,0
3,@ExplainThisBob This guy gets it,2,0
4,Sock tech is so advanced that you can get pret...,0,0
...,...,...,...
5899,@JonErlichman He’s not wrong …,0,0
5900,"@alifarhat79 Guys, I think I maybe took too mu...",0,0
5901,@sriramk Cool,0,0
5902,@cb_doge Time to complete the circle,0,0


In [ ]:
# Define a reference text for Tesla-related topics
reference_text = """
Tesla Inc. is a company that designs, manufactures, and sells electric vehicles and energy storage products.
Tesla's stock price is influenced by various factors, including company performance, innovations in electric vehicle technology,
statements and actions by CEO Elon Musk, market trends, and economic conditions.
"""

# Compute reference embedding
reference_embedding = model.encode(reference_text, convert_to_tensor=True)

# Function to analyze sentiment
def analyze_sentiment(tweet):
    blob = TextBlob(tweet)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_score = (sentiment_polarity + 1) * 5  # Convert polarity (-1 to 1) to score (1 to 10)
    return sentiment_score

# Function to analyze relevance
def compute_similarity(tweet):
    tweet_embedding = model.encode(tweet, convert_to_tensor=True)
    similarity = util.cos_sim(tweet_embedding, reference_embedding).item()
    return similarity

# Function to normalize similarity (Z-score normalization)
def normalize_similarity(similarity, mean_similarity, std_similarity):
    z_score = (similarity - mean_similarity) / std_similarity
    return z_score

# Scale absolute z-scores to range [1, 10]
def scale_to_range_1_10(z_score, min_z, max_z):
    scaled_score = ((abs(z_score) - min_z) / (max_z - min_z)) * 9 + 1
    return scaled_score

# Start analysis
start_time = time.time()

# Analyze sentiment
tweets_df['sentiment_score'] = tweets_df['text'].apply(analyze_sentiment)
sentiment_end_time = time.time()

# Format scores to two decimal places
tweets_df['sentiment_score'] = tweets_df['sentiment_score'].apply(lambda x: f"{x:.2f}")

# Print timing results
print(f"Sentiment Analysis Time: {sentiment_end_time - start_time:.2f} seconds")
#print(f"Volume Analysis Time: {volume_end_time - sentiment_end_time:.2f} seconds")
#print(f"Relevance Analysis Time: {relevance_end_time - volume_end_time:.2f} seconds")

# Save or display the results
final_tweet_df = tweets_df[['id', 'date', 'text', 'relevance_to_tesla',
                           'relevance_to_finance', 'sentiment_score']]

Sentiment Analysis Time: 1.29 seconds


### **Combine tweets and finance data**

In [ ]:
final_tweet_df = final_tweet_df.rename(columns={'date':'tweet_date'})
final_tweet_df['tweet_date'] = pd.to_datetime(final_tweet_df['tweet_date'])
final_tweet_df = final_tweet_df[final_tweet_df['tweet_date'] >= '2022-09-01']
final_tweet_df.to_csv('tweets_with_analysis_scores_2_years.csv', index=False)
final_tweet_df

,id,tweet_date,text,relevance_to_tesla,relevance_to_finance,sentiment_score
417,1567726195671797766,2022-09-08 04:07:06+00:00,You can almost hear that snap of the rubber gl...,0,0,5.00
418,1567723610197852160,2022-09-08 03:56:50+00:00,My version of Occam’s Razor: most entertaining...,2,0,6.88
419,1567722560443355141,2022-09-08 03:52:39+00:00,"“Talent hits a target no one else can hit, gen...",0,0,5.00
424,1565190122924015616,2022-09-01 04:09:39+00:00,"On a $/bot basis, this deal is awesome",0,0,10.00
425,1565189065158311937,2022-09-01 04:05:27+00:00,Sure sounds higher than 5%!\n,0,0,7.03
...,...,...,...,...,...,...
5899,1665143503108677634,2023-06-03 23:48:42+00:00,@JonErlichman He’s not wrong …,0,0,6.25
5900,1665139144425631747,2023-06-03 23:31:23+00:00,"@alifarhat79 Guys, I think I maybe took too mu...",0,0,6.00
5901,1665137204782419968,2023-06-03 23:23:41+00:00,@sriramk Cool,0,0,6.75
5902,1665131126900285445,2023-06-03 22:59:31+00:00,@cb_doge Time to complete the circle,0,0,5.50


Connect to Tesla price:

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
from datetime import datetime

# Define the symbol, period and interval for which you want to get data
# choose period between ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
# choose interval between [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

symbol = 'TSLA'
period = '2y'
Tesla_data = yf.download(tickers=symbol, period=period, interval='1h')

Tesla_data.index = pd.to_datetime(Tesla_data.index)
Tesla_data.index = Tesla_data.index.tz_convert('UTC')
Tesla_data = Tesla_data.reset_index()
Tesla_data['Close'] = Tesla_data['Close'].apply(lambda x: f"{x:.2f}")
# Tesla_data['hour'] = pd.to_datetime(Tesla_data.index).strftime('%H:%M')
# Tesla_data['date'] = pd.to_datetime(Tesla_data.index).strftime('%Y-%m-%d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
# removed volume for now
final_Tesla_data = Tesla_data[['Datetime', 'Close']]
final_tweet_df = final_tweet_df.sort_values('tweet_date')
final_tweet_df = final_tweet_df.reset_index()

final_Tesla_data = final_Tesla_data.sort_values('Datetime')
final_Tesla_data

,Datetime,Close
0,2022-08-29 14:30:00+00:00,281.65
1,2022-08-29 15:30:00+00:00,284.06
2,2022-08-29 16:30:00+00:00,285.93
3,2022-08-29 17:30:00+00:00,285.32
4,2022-08-29 18:30:00+00:00,286.17
...,...,...
3501,2024-08-28 17:30:00+00:00,203.27
3502,2024-08-28 18:30:00+00:00,205.42
3503,2024-08-28 19:30:00+00:00,205.55
3504,2024-08-29 13:30:00+00:00,211.51


In [ ]:
# find first close value in tesla data before tweet date
final_tweet_df = final_tweet_df.rename(columns={'Datetime':'tweet_date'})
final_Tesla_data = final_Tesla_data.rename(columns={'Datetime':'tesla_date'})
merged_before = pd.merge_asof(final_tweet_df, final_Tesla_data[['tesla_date', 'Close']], left_on='tweet_date', right_on='tesla_date', direction='backward')

merged_before

,level_0,index,id,tweet_date,text,relevance_to_tesla,relevance_to_finance,sentiment_score,tesla_date,Close
0,0,425,1565189065158311937,2022-09-01 04:05:27+00:00,Sure sounds higher than 5%!\n,0,0,7.03,2022-08-31 19:30:00+00:00,275.45
1,1,424,1565190122924015616,2022-09-01 04:09:39+00:00,"On a $/bot basis, this deal is awesome",0,0,10.00,2022-08-31 19:30:00+00:00,275.45
2,2,429,1565441825376243713,2022-09-01 20:49:50+00:00,@Michaelgr1011 @LAPDHQ @RickCarusoLA @ericgarc...,0,0,8.12,2022-09-01 19:30:00+00:00,277.26
3,3,430,1565884647950626816,2022-09-03 02:09:27+00:00,@heydave7 Moody’s is irrelevant,0,0,2.50,2022-09-02 19:30:00+00:00,270.23
4,4,433,1566233000458592256,2022-09-04 01:13:41+00:00,@SciGuySpace Accurate assessment. Raptor desig...,0,0,7.33,2022-09-02 19:30:00+00:00,270.23
...,...,...,...,...,...,...,...,...,...,...
5475,5475,5660,1668391031572660224,2023-06-12 22:53:13+00:00,@Liv_Boeree Hopefully,0,0,5.00,2023-06-12 19:30:00+00:00,249.88
5476,5476,5659,1668392515475587072,2023-06-12 22:59:07+00:00,@johnkrausphotos Nice shot,0,0,8.00,2023-06-12 19:30:00+00:00,249.88
5477,5477,5658,1668426637384339456,2023-06-13 01:14:42+00:00,@WallStreetSilv This doesn’t make sense,0,0,5.00,2023-06-12 19:30:00+00:00,249.88
5478,5478,5657,1668426760420048899,2023-06-13 01:15:12+00:00,@Damadeferroofic Will investigate,0,0,5.00,2023-06-12 19:30:00+00:00,249.88


In [ ]:
# find first close value in tesla data after tweet date\
merged_after = pd.merge_asof(final_tweet_df, final_Tesla_data[['tesla_date', 'Close']], left_on='tweet_date', right_on='tesla_date', direction='forward')

merged_after

,level_0,index,id,tweet_date,text,relevance_to_tesla,relevance_to_finance,sentiment_score,tesla_date,Close
0,0,425,1565189065158311937,2022-09-01 04:05:27+00:00,Sure sounds higher than 5%!\n,0,0,7.03,2022-09-01 13:30:00+00:00,273.14
1,1,424,1565190122924015616,2022-09-01 04:09:39+00:00,"On a $/bot basis, this deal is awesome",0,0,10.00,2022-09-01 13:30:00+00:00,273.14
2,2,429,1565441825376243713,2022-09-01 20:49:50+00:00,@Michaelgr1011 @LAPDHQ @RickCarusoLA @ericgarc...,0,0,8.12,2022-09-02 13:30:00+00:00,279.10
3,3,430,1565884647950626816,2022-09-03 02:09:27+00:00,@heydave7 Moody’s is irrelevant,0,0,2.50,2022-09-06 13:30:00+00:00,267.10
4,4,433,1566233000458592256,2022-09-04 01:13:41+00:00,@SciGuySpace Accurate assessment. Raptor desig...,0,0,7.33,2022-09-06 13:30:00+00:00,267.10
...,...,...,...,...,...,...,...,...,...,...
5475,5475,5660,1668391031572660224,2023-06-12 22:53:13+00:00,@Liv_Boeree Hopefully,0,0,5.00,2023-06-13 13:30:00+00:00,252.20
5476,5476,5659,1668392515475587072,2023-06-12 22:59:07+00:00,@johnkrausphotos Nice shot,0,0,8.00,2023-06-13 13:30:00+00:00,252.20
5477,5477,5658,1668426637384339456,2023-06-13 01:14:42+00:00,@WallStreetSilv This doesn’t make sense,0,0,5.00,2023-06-13 13:30:00+00:00,252.20
5478,5478,5657,1668426760420048899,2023-06-13 01:15:12+00:00,@Damadeferroofic Will investigate,0,0,5.00,2023-06-13 13:30:00+00:00,252.20


In [ ]:
merge_df=[]

In [ ]:
# merge before and after
merged_df = pd.merge(merged_before, merged_after[['tweet_date', 'tesla_date', 'Close']], on='tweet_date', suffixes=('_before','_after'))
merged_df['Close_before'] = pd.to_numeric(merged_df['Close_before'], errors='coerce')
merged_df['Close_after'] = pd.to_numeric(merged_df['Close_after'], errors='coerce')
merged_df['price_change'] = (merged_df['Close_after'] - merged_df['Close_before']).round(2)

In [ ]:
# add second price change check
merged_df['tweet_time_plus_1h'] = merged_df['tweet_date'] + pd.Timedelta(hours=1)
merged_after_1h = pd.merge_asof(merged_df[['tweet_time_plus_1h']], final_Tesla_data[['tesla_date', 'Close']], left_on='tweet_time_plus_1h', right_on='tesla_date', direction='forward')
merged_df['tesla_date_after_1h'] = merged_after_1h['tesla_date']
merged_df['Close_after_1h'] = merged_after_1h['Close']
merged_df['Close_after_1h'] = pd.to_numeric(merged_df['Close_after_1h'], errors='coerce')
merged_df['price_change_1h'] = (merged_df['Close_after_1h'] - merged_df['Close_after']).round(2)


In [ ]:
merged_df['time_diff_before'] = ((merged_df['tweet_date'] - merged_df['tesla_date_before']).dt.total_seconds() / 60.0).round(2)
merged_df['time_diff_after'] = ((merged_df['tesla_date_after'] - merged_df['tweet_date']).dt.total_seconds() / 60.0).round(2)
merged_df['time_diff_1h_after'] = ((merged_df['tesla_date_after_1h'] - merged_df['tweet_date']).dt.total_seconds() / 60.0).round(2)

merged_df

,level_0,index,id,tweet_date,text,relevance_to_tesla,relevance_to_finance,sentiment_score,tesla_date_before,Close_before,tesla_date_after,Close_after,price_change,tweet_time_plus_1h,tesla_date_after_1h,Close_after_1h,price_change_1h,time_diff_before,time_diff_after,time_diff_1h_after
0,0,425,1565189065158311937,2022-09-01 04:05:27+00:00,Sure sounds higher than 5%!\n,0,0,7.03,2022-08-31 19:30:00+00:00,275.45,2022-09-01 13:30:00+00:00,273.14,-2.31,2022-09-01 05:05:27+00:00,2022-09-01 13:30:00+00:00,273.14,0.0,515.45,564.55,564.55
1,1,424,1565190122924015616,2022-09-01 04:09:39+00:00,"On a $/bot basis, this deal is awesome",0,0,10.00,2022-08-31 19:30:00+00:00,275.45,2022-09-01 13:30:00+00:00,273.14,-2.31,2022-09-01 05:09:39+00:00,2022-09-01 13:30:00+00:00,273.14,0.0,519.65,560.35,560.35
2,2,429,1565441825376243713,2022-09-01 20:49:50+00:00,@Michaelgr1011 @LAPDHQ @RickCarusoLA @ericgarc...,0,0,8.12,2022-09-01 19:30:00+00:00,277.26,2022-09-02 13:30:00+00:00,279.10,1.84,2022-09-01 21:49:50+00:00,2022-09-02 13:30:00+00:00,279.10,0.0,79.83,1000.17,1000.17
3,3,430,1565884647950626816,2022-09-03 02:09:27+00:00,@heydave7 Moody’s is irrelevant,0,0,2.50,2022-09-02 19:30:00+00:00,270.23,2022-09-06 13:30:00+00:00,267.10,-3.13,2022-09-03 03:09:27+00:00,2022-09-06 13:30:00+00:00,267.10,0.0,399.45,5000.55,5000.55
4,4,433,1566233000458592256,2022-09-04 01:13:41+00:00,@SciGuySpace Accurate assessment. Raptor desig...,0,0,7.33,2022-09-02 19:30:00+00:00,270.23,2022-09-06 13:30:00+00:00,267.10,-3.13,2022-09-04 02:13:41+00:00,2022-09-06 13:30:00+00:00,267.10,0.0,1783.68,3616.32,3616.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5475,5475,5660,1668391031572660224,2023-06-12 22:53:13+00:00,@Liv_Boeree Hopefully,0,0,5.00,2023-06-12 19:30:00+00:00,249.88,2023-06-13 13:30:00+00:00,252.20,2.32,2023-06-12 23:53:13+00:00,2023-06-13 13:30:00+00:00,252.20,0.0,203.22,876.78,876.78
5476,5476,5659,1668392515475587072,2023-06-12 22:59:07+00:00,@johnkrausphotos Nice shot,0,0,8.00,2023-06-12 19:30:00+00:00,249.88,2023-06-13 13:30:00+00:00,252.20,2.32,2023-06-12 23:59:07+00:00,2023-06-13 13:30:00+00:00,252.20,0.0,209.12,870.88,870.88
5477,5477,5658,1668426637384339456,2023-06-13 01:14:42+00:00,@WallStreetSilv This doesn’t make sense,0,0,5.00,2023-06-12 19:30:00+00:00,249.88,2023-06-13 13:30:00+00:00,252.20,2.32,2023-06-13 02:14:42+00:00,2023-06-13 13:30:00+00:00,252.20,0.0,344.70,735.30,735.30
5478,5478,5657,1668426760420048899,2023-06-13 01:15:12+00:00,@Damadeferroofic Will investigate,0,0,5.00,2023-06-12 19:30:00+00:00,249.88,2023-06-13 13:30:00+00:00,252.20,2.32,2023-06-13 02:15:12+00:00,2023-06-13 13:30:00+00:00,252.20,0.0,345.20,734.80,734.80


In [ ]:
final_Tesla_data['date'] = final_Tesla_data['tesla_date'].dt.date
open_of_day = final_Tesla_data.groupby('date').first().reset_index()
open_of_day['time_start'] = open_of_day['tesla_date'].dt.time
open_of_day = open_of_day[['date', 'tesla_date', 'time_start', 'Close']]
open_of_day

,date,tesla_date,time_start,Close
0,2022-08-29,2022-08-29 14:30:00+00:00,14:30:00,281.65
1,2022-08-30,2022-08-30 13:30:00+00:00,13:30:00,283.57
2,2022-08-31,2022-08-31 13:30:00+00:00,13:30:00,276.38
3,2022-09-01,2022-09-01 13:30:00+00:00,13:30:00,273.14
4,2022-09-02,2022-09-02 13:30:00+00:00,13:30:00,279.10
...,...,...,...,...
499,2024-08-23,2024-08-23 13:30:00+00:00,13:30:00,220.69
500,2024-08-26,2024-08-26 13:30:00+00:00,13:30:00,212.96
501,2024-08-27,2024-08-27 13:30:00+00:00,13:30:00,209.12
502,2024-08-28,2024-08-28 13:30:00+00:00,13:30:00,207.57


In [ ]:
merged_df.columns


Index(['level_0', 'index', 'id', 'tweet_date', 'text', 'relevance_to_tesla',
       'relevance_to_finance', 'sentiment_score', 'tesla_date_before',
       'Close_before', 'tesla_date_after', 'Close_after', 'price_change',
       'tweet_time_plus_1h', 'tesla_date_after_1h', 'Close_after_1h',
       'price_change_1h', 'time_diff_before', 'time_diff_after',
       'time_diff_1h_after'],
      dtype='object')

In [ ]:
merged_df = pd.merge(merged_df, open_of_day[['date', 'tesla_date', 'Close']], how='left', left_on=merged_df['tweet_date'].dt.date, right_on='date', suffixes=('', '_first_of_day'))
merged_df = merged_df.rename(columns={'Close':'Open_value'})
merged_df = merged_df.rename(columns={'tesla_date':'Open_date'})



In [ ]:
merged_df['Open_value'] = pd.to_numeric(merged_df['Open_value'], errors='coerce')
merged_df['Close_before'] = pd.to_numeric(merged_df['Close_before'], errors='coerce')


In [ ]:
merged_df['time_diff_after_before'] =   ((merged_df['tesla_date_after'] - merged_df['tesla_date_before']).dt.total_seconds() / 60.0).round(2)


In [ ]:
# חישוב השיפוע הראשון (מהתחלת היום עד זמן הציוץ)
merged_df['slope_from_start_to_tweet'] = np.where(~pd.isna(merged_df['time_diff_from_start']),
 ((merged_df['Close_before'] - merged_df['Open_value']) / merged_df['time_diff_from_start']).round(4),
                                                  np.nan)

merged_df['slope_after_tweet'] = ((merged_df['Close_after'] - merged_df['Close_before']) / merged_df['time_diff_after_before']).round(4)



Different way to calculate slope

In [ ]:
merged_df['percent_change_before'] = ((merged_df['stock_value_before_tweet'] - merged_df['Open_stock_value']) / merged_df['Open_stock_value']) * 100
merged_df['percent_change_after'] = ((merged_df['stock_value_after_tweet'] - merged_df['stock_value_before_tweet']) / merged_df['stock_value_before_tweet']) * 100

merged_df['slope_before'] = (merged_df['percent_change_before'] / (merged_df['time_diff_from_opening to tweet'] / 60)).round(2)
merged_df['slope_after'] = (merged_df['percent_change_after'] / (merged_df['time_diff_after_before'] / 60)).round(2)


In [ ]:
merged_df.to_csv('merged_df.csv', index=False)
